Part 2 of the assigment.  Load the model and run.

In [2]:
import joblib
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
#import the sql package
from sqlalchemy import create_engine
import psycopg2
from sqlalchemy import create_engine, text

In [3]:
#set up variable for the db
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db for this class
user = r'postgres' # using the postgres user because I'm too lazy to create a new one
pw = r'BrightBlessings1!' # this is the password established during installation
port = r'5432' # default port estabalished during install
schema = r'cleaned' # schema where I last saved the data to

In [4]:
#open the connection
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [5]:
#read from the table created

table_name = r'tree'
schema = r'cleaned' # schema were the data was loaded last week.

#set df equal to disease, so we can use disease again
disease = pd.read_sql_table(table_name, db_conn, schema)
df=disease
df1=disease
df2=disease

Load the first model and run it

In [7]:
 # Load the saved model
loaded_model = joblib.load('dis1_rf_model.joblib')
print("Model loaded successfully")

#gather up names of all the columns
cols = df.columns

#set the prediction column and the feature columns for KNN
#startng with Race snce last week the 'important' feature placed race as #1
prediction_col = 'Race'
feature_cols = [c for c in cols if c != prediction_col]
X = df[feature_cols]
y = df[prediction_col]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, random_state=42, test_size=0.3, stratify=y)
# Train-Validation-Test Split - Second split
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, random_state=42, test_size=0.5, stratify=y_temp)
    # Make predictions using the loaded model
predictions = loaded_model.predict(X_test)

    # Evaluate the model (optional)
accuracy = accuracy_score(y_test, predictions)
print(f"Model accuracy: {accuracy:.2f}")

Model loaded successfully
Model accuracy: 0.41


Well, those results did not improve.  

Next, save the prediction results.
I did this as csv and then saved it to postgres.  I'm sure I could have skipped the csv.

In [9]:
dfa = pd.DataFrame(predictions)
dfa.index = X_test.index # its important for comparison
dfa.columns = ["prediction"]
dfa.to_csv("prediction_results.csv")

In [10]:
table_name = r'dis1_pred'
schema = r'analytics' # schema we created this week.

dfa.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

87

load the second model

In [12]:
loaded_model = joblib.load('dis2_rf_model.joblib')
print("Model loaded successfully")

#gather up names of all the columns
cols = df1.columns

#set the prediction column and the feature columns for KNN
#startng with Race snce last week the 'important' feature placed race as #1
prediction1_col = 'gender'
feature1_cols = [c for c in cols if c != prediction1_col]
X = df1[feature1_cols]
y = df1[prediction1_col]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, random_state=42, test_size=0.3, stratify=y)
# Train-Validation-Test Split - Second split
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, random_state=42, test_size=0.5, stratify=y_temp)
    # Make predictions using the loaded model
predictions1 = loaded_model.predict(X_test)

    # Evaluate the model (optional)
accuracy = accuracy_score(y_test, predictions1)
print(f"Model accuracy: {accuracy:.2f}")

Model loaded successfully
Model accuracy: 0.84


Using gender improves the accuracy.
save the predictions out to csv then to the db

In [14]:
dfb = pd.DataFrame(predictions1)
dfb.index = X_test.index # its important for comparison
dfb.columns = ["prediction1"]
dfb.to_csv("prediction1_results.csv")

In [15]:
table_name = r'dis2_pred'
schema = r'analytics' # schema we created this week.

dfb.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

87

Load and run the last model

In [17]:
loaded_model = joblib.load('dis3_rf_model.joblib')
print("Model loaded successfully")

#gather up names of all the columns
cols = df2.columns

#set the prediction column and the feature columns for KNN
#startng with Race snce last week the 'important' feature placed race as #1
prediction2_col = 'Age'
feature2_cols = [c for c in cols if c != prediction2_col]
X = df2[feature2_cols]
y = df2[prediction2_col]
X_train, X_temp, y_train, y_temp = train_test_split(X, y, random_state=42, test_size=0.3, stratify=y)
# Train-Validation-Test Split - Second split
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, random_state=42, test_size=0.5, stratify=y_temp)
    # Make predictions using the loaded model
predictions2 = loaded_model.predict(X_test)

    # Evaluate the model (optional)
accuracy = accuracy_score(y_test, predictions2)
print(f"Model accuracy: {accuracy:.2f}")

Model loaded successfully
Model accuracy: 0.94


Of the 3 features chosen, this one performed the best
save the prediction to csv and then to the db.

In [19]:
dfc = pd.DataFrame(predictions2)
dfc.index = X_test.index # its important for comparison
dfc.columns = ["prediction2"]
dfc.to_csv("prediction2_results.csv")

In [20]:
table_name = r'dis3_pred'
schema = r'analytics' # schema we created this week.

dfc.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

87

This part was challenging in getting the model to work in a separate notebook and then saving the prediction.  

References used:
<p> https://www.analyticsvidhya.com/blog/2023/02/how-to-save-and-load-machine-learning-models-in-python-using-joblib-library/
<p>https://stackoverflow.com/questions/34864695/saving-prediction-results-to-csv
<p>https://plainenglish.io/blog/importing-csv-data-into-postgresql-using-python